In [19]:
taskname = input()

 dsa


## Run this before Mask

In [1]:
import os

### Create folder for masks

In [2]:
os.chdir(os.path.dirname(os.getcwd()))
MASK_DIR = os.path.join(os.getcwd(), 'masks')
MASK_DIR

'/scratch/gpfs/ziyangc/pilus/Pilus_Sensing/data/221214/different_strain_arabinose/masks'

In [3]:
if not os.path.exists(MASK_DIR):
    os.mkdir(MASK_DIR)
    os.mkdir(os.path.join(MASK_DIR, 'preview'))
    os.mkdir(os.path.join(MASK_DIR, 'masks_all'))

In [ ]:
FLUO_DIR = os.path.join(os.getcwd(), 'fluo')
if not os.path.exists(FLUO_DIR):
    os.mkdir(FLUO_DIR)

### Generate args for masks

In [4]:
ref_dir = os.path.join(os.getcwd(), 'ref.txt')
print(os.path.exists(ref_dir))
ref = {}

True


In [5]:
for line in open(ref_dir):
    a = line.strip()
    if(a[0] == '#'):
        continue
    a = a.split(',')
    ref[a[0]] = a[1:]
    
print(ref)

{'strain': ['12++', '12+-', '12-+', '12--', '1++', '1+-', '1-+', '1--', '2++', '2+-', '2-+', '2--'], 'conc': ['1.5'], 'tp': ['0', '30', '60', '90', '120', '150', '180', '210', '240'], 'pos': ['0', '1', '2', '3', '4', '5'], 'od': ['.1', '.1', '.1', '.1'], 'ara_conc': ['1']}


In [6]:
a1 = len(ref['strain'])
a2 = len(ref['pos'])

In [8]:
print("[%d-%d:%d]" % (0, a1*a2, a2))

[0-72:6]


### Create Slurm for FindAllMasks

In [22]:
slurm_dir = os.path.join(os.getcwd(), 'analysis', 'Slurm1.slurm')
slurm_dir

'/scratch/gpfs/ziyangc/pilus/Pilus_Sensing/data/221214/different_strain_arabinose/analysis/Slurm1.slurm'

In [23]:
with open(slurm_dir, 'w') as f:
    f.write("#!/bin/bash\n")
    f.write("#SBATCH --job-name={}     # create a short name for your job\n".format(taskname))
    f.write("#SBATCH --output=./out/%x-%a.out # stdout file\n")
    f.write("#SBATCH --error=./err/%x-%a.err  # stderr file\n")
    f.write("#SBATCH --nodes=1                # node count\n")
    f.write("#SBATCH --ntasks=1               # total number of tasks across all nodes\n")
    f.write("#SBATCH --cpus-per-task=12        # cpu-cores per task (>1 if multi-threaded tasks)\n")
    f.write("#SBATCH --mem=1G         # memory\n")
    f.write("#SBATCH --time=02:30:00          # total run time limit (HH:MM:SS)\n")
    f.write("#SBATCH --array=[0-{}:{}]             # job array with index values 0, 1, 2, 3, 4\n".format(a1*a2, a2))
    f.write("#SBATCH --mail-type=all          # send email on job start, end and fault\n")
    f.write("module purge\n")
    f.write("module load anaconda3/2021.11\n")
    f.write("conda activate pilus\n")
    f.write("python GetAllMasks.py $SLURM_ARRAY_TASK_ID {}\n".format(a2))

In [ ]:
err_dir = os.path.join(os.getcwd(), 'analysis', 'err')
out_dir = os.path.join(os.getcwd(), 'analysis', 'out')
prog_dir = os.path.join(os.getcwd(), 'analysis', 'progress')
if not os.path.exists(err_dir):
    os.mkdir(err_dir)
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
if not os.path.exists(prog_dir):
    os.mkdir(prog_dir)